<div style="text-align: right" align="right"><i>Peter Norvig, 3 Jan 2020</i></div>

# Spelling Bee Puzzle

The [3 Jan. 2020 edition of the 538 Riddler](https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/) concerns the popular NYTimes  [Spelling Bee](https://www.nytimes.com/puzzles/spelling-bee) puzzle:

> In this game, seven letters are arranged in a **honeycomb** lattice, with one letter in the center. Here’s the lattice from Dec. 24, 2019:
> 
> <img src="https://fivethirtyeight.com/wp-content/uploads/2020/01/Screen-Shot-2019-12-24-at-5.46.55-PM.png?w=1136" width="150">
> 
> The goal is to identify as many words as possible that meet the following criteria:
> 1. The word must be at least four letters long.
> 2. The word must include the central letter.
> 3. The word cannot include any letter beyond the seven given letters.
>
>Note that letters can be repeated. For example, the words GAME and AMALGAM are both acceptable words. Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, seven-letter words are worth 7 points, etc. Words that use all of the seven letters in the honeycomb are known as "**pangrams**" and earn 7 bonus points (in addition to the points for the length of the word). So in the above example, MEGAPLEX is worth 15 points.
>
> ***Which seven-letter honeycomb results in the highest possible game score?*** To be a valid choice of seven letters, no letter can be repeated, it must not contain the letter S (that would be too easy) and there must be at least one pangram.
>
> For consistency, please use [this word list](https://norvig.com/ngrams/enable1.txt) to check your game score.



Since the referenced [word list](https://norvig.com/ngrams/enable1.txt) came from *my* web site, I felt somewhat compelled to solve this one.  (Note I didn't make up the word list; it is a standard Scrabble word list that I happen to host a copy of.) I'll show you how I address the problem, step by step:

# Step 1: Words, Word Scores, and Pangrams

Let's start by defining some basics:

- A **valid word** is a string of at least 4 letters, with no 'S', and not more than 7 distinct letters.
- A **word list** is, well, a list of words.
- A **pangram** is a word with exactly 7 distinct letters; it scores a **pangram bonus** of 7 points.
- The **word score** is 1 for a four letter word, or the length of the word for longer words, plus any pangram bonus.


In [1]:
from typing      import List, Set, Tuple, Dict
from collections import Counter, defaultdict, namedtuple
from itertools   import combinations

In [2]:
Word = str # Type for a word

def valid_words(text) -> List[Word]:
    """Words with at least 4 letters, no 'S', and no more than 7 distinct letters."""
    return [w for w in text.upper().split() 
            if len(w) >= 4 and 'S' not in w and len(set(w)) <= 7]

def pangram_bonus(word) -> int: 
    """Does a word get a bonus for having 7 distinct letters (some maybe more than once)?"""
    return 7 if len(set(word)) == 7 else 0

def word_score(word) -> int: 
    """The points for this word, including bonus for pangram."""
    return 1 if len(word) == 4 else len(word) + pangram_bonus(word)

I'll make a mini word list to experiment with: 

In [3]:
mini = valid_words('game amalgam amalgamation glam gem gems em megaplex cacciatore erotica')
mini

['GAME', 'AMALGAM', 'GLAM', 'MEGAPLEX', 'CACCIATORE', 'EROTICA']

Note that `gem` and `em` are too short, `gems` has an `s` which is not allowed, and `amalgamation` has too many distinct letters (8). We're left with six valid words out of the ten candidate words. Here are examples of the other two functions in action:

In [4]:
{w for w in mini if pangram_bonus(w)}

{'CACCIATORE', 'EROTICA', 'MEGAPLEX'}

In [5]:
{w: word_score(w) for w in mini}

{'GAME': 1,
 'AMALGAM': 7,
 'GLAM': 1,
 'MEGAPLEX': 15,
 'CACCIATORE': 17,
 'EROTICA': 14}

# Step 2: Honeycombs and Game Scores

In a honeycomb the order of the letters doesn't matter; all that matters is:
  1. The seven distinct letters in the honeycomb.
  2. The one distinguished center letter.
  
Thus, we can represent a honeycomb as follows:
  

In [6]:
Honeycomb = namedtuple('Honeycomb', 'letters, center')

hc = Honeycomb('AEGLMPX', 'G')
hc

Honeycomb(letters='AEGLMPX', center='G')

The **game score** for a honeycomb is the sum of the word scores for all the words that the honeycomb can make. How do we know if a honeycomb can make a word? It can if (1) the word contains the honeycomb's center and (2) every letter in the word is in the honeycomb. 

In [7]:
def game_score(honeycomb, wordlist) -> int:
    """The total score for this honeycomb."""
    return sum(word_score(w) 
               for w in wordlist if can_make(honeycomb, w))

def can_make(honeycomb, word) -> bool:
    """Can the honeycomb make this word?"""
    letters, center = honeycomb
    return center in word and all(L in letters for L in word)

In [8]:
game_score(hc, mini)

24

In [9]:
{w: word_score(w) for w in mini if can_make(hc, w)}

{'GAME': 1, 'AMALGAM': 7, 'GLAM': 1, 'MEGAPLEX': 15}

# Step 3: Best Honeycomb


How many possible honeycombs are there? We can put any letter in the center, then any 6 letters around the outside; since there is no 'S', this gives a total of 25 × (24 choose 6) = 3,364,900 possible honeycombs. We could conceivably ask for the game score of every one of them and pick the best; that would probably take hours of computation (not seconds, and not days).

However, a key constraint of the game is that **there must be at least one pangram** in the set of words that a valid honeycomb can make. That means that a valid honeycomb must ***be*** the set of seven letters in one of the pangram words in the word list, with any of the seven letters as the center. My approach to find the best (highest scoring) honeycomb is:

  * Go through all the words and find all the valid honeycombs: the 7-letter pangram letter sets, with any of the 7 letters as center.
  * Compute the game score for each valid honeycomb and return a honeycomb with maximal game score.

In [10]:
def best_honeycomb(words) -> Honeycomb: 
    """Return a honeycomb with highest game score on these words."""
    return max(valid_honeycombs(words), 
               key=lambda h: game_score(h, words))

def valid_honeycombs(words) -> List[Honeycomb]:
    """Valid Honeycombs are the pangram lettersets, with any center."""
    pangram_lettersets = {letterset(w) for w in words if pangram_bonus(w)}
    return [Honeycomb(letters, center) 
            for letters in pangram_lettersets 
            for center in letters]

I will represent a set of letters as a sorted string of distinct letters. Why not a Python `set` (or `frozenset` if we want it to be the key of a dict)? Because a string takes up less space in memory, and its printed representation is  easier to read when debugging. Compare:
- `frozenset({'A', 'E', 'G', 'L', 'M', 'P', 'X'})`
- `'AEGLMPX'`

I'll use the name `letterset` for the function that converts a word to a set of letters, and `Letterset` for the resulting type:

In [11]:
Letterset = str # Type for a set of letters, like "AGLM"

def letterset(word) -> Letterset:
    """The set of letters in a word, represented as a sorted str."""
    return ''.join(sorted(set(word)))

In [12]:
{w: letterset(w) for w in mini}

{'GAME': 'AEGM',
 'AMALGAM': 'AGLM',
 'GLAM': 'AGLM',
 'MEGAPLEX': 'AEGLMPX',
 'CACCIATORE': 'ACEIORT',
 'EROTICA': 'ACEIORT'}

Note that 'AMALGAM' and 'GLAM' have the same letterset, as do 'CACCIATORE' and 'EROTICA'.

In [13]:
valid_honeycombs(mini)

[Honeycomb(letters='ACEIORT', center='A'),
 Honeycomb(letters='ACEIORT', center='C'),
 Honeycomb(letters='ACEIORT', center='E'),
 Honeycomb(letters='ACEIORT', center='I'),
 Honeycomb(letters='ACEIORT', center='O'),
 Honeycomb(letters='ACEIORT', center='R'),
 Honeycomb(letters='ACEIORT', center='T'),
 Honeycomb(letters='AEGLMPX', center='A'),
 Honeycomb(letters='AEGLMPX', center='E'),
 Honeycomb(letters='AEGLMPX', center='G'),
 Honeycomb(letters='AEGLMPX', center='L'),
 Honeycomb(letters='AEGLMPX', center='M'),
 Honeycomb(letters='AEGLMPX', center='P'),
 Honeycomb(letters='AEGLMPX', center='X')]

In [14]:
best_honeycomb(mini)

Honeycomb(letters='ACEIORT', center='A')

**We're done!** We know how to find the best honeycomb. But so far, we've only done it  for the mini word list. 

# Step 4: The enable1 Word List

Here's the real word list, `enable1.txt`, and some counts derived from it:

In [15]:
! [ -e enable1.txt ] || curl -O http://norvig.com/ngrams/enable1.txt
! wc -w enable1.txt

  172820 enable1.txt


In [16]:
enable1 = valid_words(open('enable1.txt').read())
len(enable1)

44585

In [17]:
pangrams = [w for w in enable1 if pangram_bonus(w)]
len(pangrams)

14741

In [18]:
len({letterset(w) for w in pangrams}) # pangram lettersets

7986

In [19]:
len(valid_honeycombs(enable1))

55902

To summarize, there are:

- 172,820 words in the `enable1` word list
- 44,585 valid Spelling Bee words
- 14,741 pangram words 
- 7,986 distinct pangram lettersets
- 55,902 (7 × 7,986) valid honeycombs

How long will it take to run `best_honeycomb(enable1)`? Most of the computation time is in `game_score` (which has to look at all 44,585 valid words), so let's estimate the total time by first checking how long it takes to compute the game score of a single honeycomb:

In [20]:
%time game_score(hc, enable1)

CPU times: user 9.77 ms, sys: 35 µs, total: 9.8 ms
Wall time: 9.8 ms


153

About 10 milliseconds on my computer (this may vary). How many minutes would it be to run `game_score` for all 55,902 valid honeycombs?

In [21]:
.01 * 55902 / 60

9.317

About 9 or 10 minutes. I could run `best_honeycomb(enable1)` right now and take a coffee break until it completes, but I think that a puzzle like this deserves a more elegant solution. I'd like to get the run time under a minute (as is suggested in [Project Euler](https://projecteuler.net/)), and I have an idea how to do it.

# Step 5: Faster Algorithm: Points Table

Here's my plan for a more efficient program:

1. Keep the same strategy of trying every pangram letterset, but do some precomputation that will make `game_score` much faster.
1. The precomputation is: compute the `letterset` and `word_score` for each word, and make a table of `{letterset: total_points}` giving the total number of word score points for all the words that correspond to each letterset. I call this a **points table**.
3. These calculations are independent of the honeycomb, so they need to be done only once, not 55,902  times. 
4. `game_score2` (the name is changed because the interface has changed) takes a honeycomb and a points table as input. The idea is that every word that the honeycomb can make must have a letterset that is the same as a valid **letter subset** of the honeycomb. A valid letter subset must include the center letter, and it may or may not include each of the other 6 letters, so there are exactly $2^6 = 64$ valid letter subsets. (The function `letter_subsets(honeycomb)` computes these.)
The result of `game_score2` is the sum of the honeycomb's 64 letter subset entries in the points table.


That means that in `game_score2` we no longer need to iterate over 44,585 words and check if each word is a subset of the honeycomb. Instead we iterate over the 64 subsets of the honeycomb and for each one check&mdash;in one table lookup&mdash;whether it is a word (or more than word) and how many total points those word(s) score.  Since 64 &lt; 44,585, that's a nice optimization!


Here's the code:

In [22]:
PointsTable = Dict[Letterset, int]

def best_honeycomb(words) -> Honeycomb: 
    """Return a honeycomb with highest game score on these words."""
    points_table = tabulate_points(words)
    honeycombs = (Honeycomb(letters, center) 
                  for letters in points_table if len(letters) == 7 
                  for center in letters)
    return max(honeycombs, key=lambda h: game_score2(h, points_table))

def tabulate_points(words) -> PointsTable:
    """Return a Counter of {letterset: points} from words."""
    table = Counter()
    for w in words:
        table[letterset(w)] += word_score(w)
    return table

def letter_subsets(honeycomb) -> List[Letterset]:
    """The 64 subsets of the letters in the honeycomb, always including the center letter."""
    return [letters 
            for n in range(1, 8) 
            for letters in map(''.join, combinations(honeycomb.letters, n))
            if honeycomb.center in letters]

def game_score2(honeycomb, points_table) -> int:
    """The total score for this honeycomb, using a points table."""
    return sum(points_table[letterset] for letterset in letter_subsets(honeycomb))

Let's get a feel for how this works. 

First `letter_subsets` (a 4-letter honeycomb makes $2^3 = 8$ subsets; 7-letter honeycombs make 64):

In [23]:
letter_subsets(Honeycomb('ABCD', 'C')) 

['C', 'AC', 'BC', 'CD', 'ABC', 'ACD', 'BCD', 'ABCD']

In [24]:
mini # Remind me again what the mini word list is?

['GAME', 'AMALGAM', 'GLAM', 'MEGAPLEX', 'CACCIATORE', 'EROTICA']

Now `tabulate_points`:

In [25]:
tabulate_points(mini)

Counter({'AEGM': 1, 'AGLM': 8, 'AEGLMPX': 15, 'ACEIORT': 31})

The letterset `'AGLM'` gets 8 points, 7 for AMALGAM and 1 for GLAM.  `'ACEIORT'` gets 31 points, 17 for CACCIATORE and 14 for EROTICA. The other lettersets represent one word each. 

Let's make sure we haven't broken the  `best_honeycomb` function:

In [26]:
assert best_honeycomb(mini).letters == 'ACEIORT'

# Step 6: The Solution

Finally, the solution to the puzzle:

In [27]:
%time best = best_honeycomb(enable1)

best, game_score(best, enable1)

CPU times: user 1.81 s, sys: 2.31 ms, total: 1.82 s
Wall time: 1.82 s


(Honeycomb(letters='AEGINRT', center='R'), 3898)

**Wow! 3898 is a high score!** 

And it took less than 2 seconds of computation to find the best honeycomb!

# Step 7: Even Faster Algorithm: Branch and Bound

A run time of 2 seconds is pretty good! But what if the word list were 100 times bigger? What if a honeycomb had 12 letters around the outside, not just 6? We might still be looking for ideas to speed up the computation.  I happen to have one.

Consider the word 'EQUIVOKE'. It is a pangram, but what with the 'Q' and 'V' and 'K', it is not a high-scoring honeycomb, regardless of what center is used:

In [28]:
{C: game_score(Honeycomb('EIKOQUV', C), enable1)
    for C in 'EQUIVOK'}

{'E': 48, 'Q': 29, 'U': 29, 'I': 32, 'V': 35, 'O': 36, 'K': 34}

It would be great if we could eliminate all seven of these honeycombs at once, rather than trying each one in turn. So my idea is to:
- Keep track of the best honeycomb and best score found so far.
- For each new pangram letterset, ask "if we weren't required to use the center letter, would this letterset score higher than the best honeycomb so far?" 
- If yes, then try it with all seven centers; if not then discard it immediately.
- This is called a [**branch and bound**](https://en.wikipedia.org/wiki/Branch_and_bound) algorithm: if an **upper bound** of the new letterset's score can't beat the best honeycomb so far, then we prune a whole **branch** of the search tree consisting of the seven honeycombs that have that letterset.

What would the score of a letterset be if we weren't required to use the center letter? It turns out I can make a dummy Honeycomb and specify the empty string for the center, `Honeycomb(letters, '')`, and call `game_score2` on that. This works because of a quirk of Python: we ask if `honeycomb.center in letters`; normally in Python the expression `x in y` means "is `x` a member of the collection `y`", but when `y` is a string it means "is `x` a substring of `y`", and the empty string is a substring of every string. (If I had represented a letterset as a Python `set`, this wouldn't work.)

Thus, I can rewrite `best_honeycomb` as follows:

In [29]:
def best_honeycomb(words) -> Honeycomb: 
    """Return a honeycomb with highest game score on these words."""
    points_table = tabulate_points(words)
    best, best_score = None, 0
    pangrams = (s for s in points_table if len(s) == 7)
    for p in pangrams:
        if game_score2(Honeycomb(p, ''), points_table) > best_score:
            for center in p:
                honeycomb = Honeycomb(p, center)
                score = game_score2(honeycomb, points_table)
                if score > best_score:
                    best, best_score = honeycomb, score
    return best

In [30]:
%time best_honeycomb(enable1)

CPU times: user 406 ms, sys: 1.13 ms, total: 407 ms
Wall time: 406 ms


Honeycomb(letters='AEGINRT', center='R')

Same honeycomb for the answer, but four times faster&mdash;less than half a second.

# Step 8: Curiosity

I'm curious about a bunch of things.

### What's the highest-scoring individual word?

In [31]:
max(enable1, key=word_score)

'ANTITOTALITARIAN'

### What are some of the pangrams?

In [32]:
pangrams[::500] # Every five-hundreth pangram

['AARDWOLF',
 'ANCIENTER',
 'BABBLEMENT',
 'BIVARIATE',
 'CABEZON',
 'CHEERFUL',
 'COLLOGUING',
 'CRANKLE',
 'DEMERGERING',
 'DWELLING',
 'ETYMOLOGY',
 'FLATTING',
 'GARROTTING',
 'HANDIER',
 'IDENTIFY',
 'INTERVIEWER',
 'LARVICIDAL',
 'MANDRAGORA',
 'MORTGAGEE',
 'NOTABLE',
 'OVERHELD',
 'PERONEAL',
 'PRAWNED',
 'QUILTER',
 'REINITIATED',
 'TABLEFUL',
 'TOWHEAD',
 'UNCHURCHLY',
 'UTOPIAN',
 'WINDAGE']

### What's the breakdown of reasons why words are invalid?

In [33]:
Counter('has S' if 'S' in w else 
        '< 4'   if len(w) < 4 else 
        '> 7'   if len(set(w)) > 7 else 
        'valid'
        for w in open('enable1.txt').read().upper().split()).most_common()

[('has S', 103913), ('valid', 44585), ('> 7', 23400), ('< 4', 922)]

There are more than twice as many words with an 'S' as there are valid words.

### About the points table: How many different letter subsets are there? 

In [34]:
pts = tabulate_points(enable1)
len(pts)

21661

That means there's about two valid words for each letterset.

### Which letter subsets score the most?

In [35]:
pts.most_common(10)

[('AEGINRT', 832),
 ('ADEGINR', 486),
 ('ACILNOT', 470),
 ('ACEINRT', 465),
 ('CEINORT', 398),
 ('AEGILNT', 392),
 ('AGINORT', 380),
 ('ADEINRT', 318),
 ('CENORTU', 318),
 ('ACDEIRT', 307)]

The best honeycomb is also the highest scoring letter subset on its own (although it only gets 832 of the 3,898 total points from using all seven letters).

# Step 9: Fancy Report

I'd like to see the actual words that each honeycomb can make, in addition to the total score, and I'm curious about how the words are divided up by letterset. Here's a function to provide such a report. I remembered that there is a `fill` function in Python (it is in the `textwrap` module) but this turned out to be a lot more complicated than I expected. I guess it is difficult to create a practical extraction and reporting tool. I feel you, [Larry Wall](http://www.wall.org/~larry/).

In [36]:
from textwrap import fill

def report(words, honeycomb=None):
    """Print stats, words, and word scores for the given honeycomb (or 
    for the best honeycomb if no honeycomb is given) over the given word list."""
    bins    = group_by(words, letterset)
    adj     = ("given" if honeycomb else "optimal")
    honeycomb = honeycomb or best_honeycomb(words)
    points  = game_score(honeycomb, words)
    subsets = letter_subsets(honeycomb)
    nwords  = sum(len(bins[s]) for s in subsets)
    print(f'For this list of {Ns(len(words), "word")}:')
    print(f'The {adj} honeycomb {honeycomb.letters} with center {honeycomb.center}'
          f' makes {Ns(nwords, "word")} for {Ns(points, "point")}:\n')
    indent = ' ' * 4
    for s in sorted(subsets, key=lambda s: (-len(s), s)):
        if bins[s]:
            pts = sum(word_score(w) for w in bins[s])
            print(f'{s} makes {Ns(len(bins[s]), "word")} for {Ns(pts, "point")}:')
            words = [f'{w}({word_score(w)})' for w in sorted(bins[s])]
            print(fill(' '.join(words), width=80,
                       initial_indent=indent, subsequent_indent=indent))
            
def Ns(n, noun):
    """A string with `n` followed by the plural or singular of noun:
    Ns(3, 'bear') => '3 bears'; Ns(1, 'world') => '1 world'"""  
    return f"{n:,d} {noun}{'' if n == 1 else 's'}"

def group_by(items, key):
    "Group items into bins of a dict, each bin keyed by key(item)."
    bins = defaultdict(list)
    for item in items:
        bins[key(item)].append(item)
    return bins

In [37]:
report(mini, hc)

For this list of 6 words:
The given honeycomb AEGLMPX with center G makes 4 words for 24 points:

AEGLMPX makes 1 word for 15 points:
    MEGAPLEX(15)
AEGM makes 1 word for 1 point:
    GAME(1)
AGLM makes 2 words for 8 points:
    AMALGAM(7) GLAM(1)


In [38]:
report(enable1)

For this list of 44,585 words:
The optimal honeycomb AEGINRT with center R makes 537 words for 3,898 points:

AEGINRT makes 50 words for 832 points:
    AERATING(15) AGGREGATING(18) ARGENTINE(16) ARGENTITE(16) ENTERTAINING(19)
    ENTRAINING(17) ENTREATING(17) GARNIERITE(17) GARTERING(16) GENERATING(17)
    GNATTIER(15) GRANITE(14) GRATINE(14) GRATINEE(15) GRATINEEING(18)
    GREATENING(17) INGRATE(14) INGRATIATE(17) INTEGRATE(16) INTEGRATING(18)
    INTENERATING(19) INTERAGE(15) INTERGANG(16) INTERREGNA(17) INTREATING(17)
    ITERATING(16) ITINERATING(18) NATTERING(16) RATTENING(16) REAGGREGATING(20)
    REATTAINING(18) REGENERATING(19) REGRANTING(17) REGRATING(16)
    REINITIATING(19) REINTEGRATE(18) REINTEGRATING(20) REITERATING(18)
    RETAGGING(16) RETAINING(16) RETARGETING(18) RETEARING(16) RETRAINING(17)
    RETREATING(17) TANGERINE(16) TANGIER(14) TARGETING(16) TATTERING(16)
    TEARING(14) TREATING(15)
AEGINR makes 35 words for 270 points:
    AGINNER(7) AGREEING(8) ANEARING(8

# Step 10: S Words

What if we allowed honeycombs and words to have an 'S' in them?

In [39]:
enable1s = [w for w in open('enable1.txt').read().upper().split() 
           if len(w) >= 4 and len(set(w)) <= 7]

len(enable1s), len(enable1)

(98141, 44585)

That more than doubles the number of words. Will it double the score?

In [40]:
report(enable1s)

For this list of 98,141 words:
The optimal honeycomb AEINRST with center E makes 1,179 words for 8,681 points:

AEINRST makes 86 words for 1,381 points:
    ANESTRI(14) ANTISERA(15) ANTISTRESS(17) ANTSIER(14) ARENITES(15)
    ARSENITE(15) ARSENITES(16) ARTINESS(15) ARTINESSES(17) ATTAINERS(16)
    ENTERTAINERS(19) ENTERTAINS(17) ENTRAINERS(17) ENTRAINS(15) ENTREATIES(17)
    ERRANTRIES(17) INERTIAS(15) INSTANTER(16) INTENERATES(18) INTERSTATE(17)
    INTERSTATES(18) INTERSTRAIN(18) INTERSTRAINS(19) INTRASTATE(17) INTREATS(15)
    IRATENESS(16) IRATENESSES(18) ITINERANTS(17) ITINERARIES(18) ITINERATES(17)
    NASTIER(14) NITRATES(15) RAINIEST(15) RATANIES(15) RATINES(14) REATTAINS(16)
    REINITIATES(18) REINSTATE(16) REINSTATES(17) RESINATE(15) RESINATES(16)
    RESISTANT(16) RESISTANTS(17) RESTRAIN(15) RESTRAINER(17) RESTRAINERS(18)
    RESTRAINS(16) RESTRAINT(16) RESTRAINTS(17) RETAINERS(16) RETAINS(14)
    RETINAS(14) RETIRANTS(16) RETRAINS(15) RETSINA(14) RETSINAS(15)
    SANITARIE

Yes it does  (roughly) double the score!

# Summary

Thanks to a series of ideas, we were able to achieve a substantial reduction in the number of honeycombs that need to be examined (a factor of 400), the run time needed for `game_score` (a factor of about 200), and the overall run time (a factor of about 70,000).

- **Enumeration (10 hours (estimate) run time; 3,364,900 honeycombs)**<br>Try every possible honeycomb.
- **Pangram Lettersets (10 minutes (estimate) run time; 55,902 honeycombs)**<br>Try just the honeycombs that are pangram lettersets (with every center).
- **Points Table (under 2 seconds run time; 55,902 honeycombs)**<br>Precompute the score for each letterset, and sum the 64 letter subsets of each honeycomb.
- **Branch and Bound (under 1/2 second run time; 8,084 honeycombs)**<br>Try every center only for lettersets that score better than the best score so far.



Here are pictures for the highest-scoring honeycombs, with and without an S:

<img src="http://norvig.com/honeycombs.png" width="350">
<center>
   537 words; 3,898 points &nbsp; &nbsp; &nbsp; &nbsp; 1,179 words; 8,681 points
    <br>
</center>